<a href="https://colab.research.google.com/github/TimHBSWFL/UCSD-ML-Capstone/blob/main/hyperparameter_tuning_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00


In [15]:
import os
import torch
import optuna
import numpy as np
from torch import nn
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
directory = '/content/drive/My Drive/Capstone Data Collection/Image Datasets'

dataframes = {}

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        dataframes[filename] = df

for key, df in dataframes.items():
    print(f"{key}: {df.shape}")

Images_to_Text_Results.csv: (150, 3)
df_upscale_photos.csv: (796, 18)
lower_rated.csv: (333, 21)
highly_rated.csv: (3760, 21)
middle_rated.csv: (918, 21)


In [7]:
df_captions = dataframes['Images_to_Text_Results.csv']

df_list = ['highly_rated.csv', 'middle_rated.csv', 'lower_rated.csv', 'df_upscale_photos.csv']

df_photos = []

for i in df_list:
  if i in dataframes:
    df = dataframes[i]
    df_photos.append(df)

df_concat = pd.concat(df_photos, ignore_index=True)
df_concat.shape

(5807, 21)

In [8]:
df_sliced = df_concat[['photo_id', 'label']]

df_captions['photo_id'] = df_captions['photo_id'].str.replace('.jpg', '', regex=False)

df_merged = df_captions.merge(df_sliced, how='left', on='photo_id')
df_merged.shape

(150, 4)

In [9]:
df_merged['photo_id'] = df_merged['photo_id'] + '.jpg'

df_merged['label'].value_counts()

,count
label,
fast food,75
fine dining,75


In [10]:
image_dir = "/content/drive/My Drive/Capstone Data Collection/test photos"

image_paths = []

for subfolder in ['fast food', 'upscale']:
    subfolder_path = os.path.join(image_dir, subfolder)

    image_paths.extend([os.path.join(subfolder_path, img) for img in os.listdir(subfolder_path) if img.endswith(".jpg")])

print(len(image_paths))

50


In [11]:
df_merged['labels'] = df_merged['label'].map({"fast food": 0, "fine dining": 1})
df_merged.head()

,photo_id,model_name,caption,label,labels
0,Ax5PLwfU94uEXMafFdXrtw.jpg,nlpconnect/vit-gpt2-image-captioning,a sandwich with meat and cheese on a cutting b...,fast food,0
1,Ax5PLwfU94uEXMafFdXrtw.jpg,Salesforce/blip-image-captioning-large,a close up of a sandwich on a piece of paper o...,fast food,0
2,Ax5PLwfU94uEXMafFdXrtw.jpg,Salesforce/blip-image-captioning-base,"a hamburger with cheese, bacon and cheese on it",fast food,0
3,bFNqVruIW3AXjgSuLHq4kg.jpg,nlpconnect/vit-gpt2-image-captioning,a person holding a sandwich in their hand,fast food,0
4,bFNqVruIW3AXjgSuLHq4kg.jpg,Salesforce/blip-image-captioning-large,someone is grabbing a sausage patty out of a b...,fast food,0


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
image_features = []
text_features = []
labels = []


for idx, row in df_merged.iterrows():
  target_filename = row["photo_id"] if row["photo_id"].endswith(".jpg") else f"{row['photo_id']}.jpg"
  image_path = next((path for path in image_paths if os.path.basename(path) == target_filename), None)

  image = Image.open(image_path).convert("RGB")
  caption = row["caption"]
  label = row["labels"]

  inputs = clip_processor(text=[caption], images=image, return_tensors="pt", padding=True).to(device)
  with torch.no_grad():
    image_embedding = clip_model.get_image_features(pixel_values=inputs['pixel_values']).squeeze().cpu()
    text_embedding = clip_model.get_text_features(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask']).squeeze().cpu()

  image_features.append(image_embedding)
  text_features.append(text_embedding)
  labels.append(label)

image_features = torch.stack(image_features)
text_features = torch.stack(text_features)
labels = torch.tensor(labels)

In [17]:

class MultimodalClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate):
        super(MultimodalClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim * 2, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, image_embedding, text_embedding):
        x = torch.cat((image_embedding, text_embedding), dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        return x


def objective(trial):
    lr = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    epochs = trial.suggest_int("epochs", 10, 20)
    weight_decay = trial.suggest_loguniform("weight_decay", 0.01, 0.1)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)

    X_train, X_val, y_train, y_val = train_test_split(image_features, labels, test_size=0.2, random_state=42)
    text_train, text_val = train_test_split(text_features, test_size=0.2, random_state=42)

    train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(text_train), torch.tensor(y_train))
    val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(text_val), torch.tensor(y_val))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    input_dim = image_features.shape[1]
    hidden_dim = 256
    model = MultimodalClassifier(input_dim=input_dim, hidden_dim=hidden_dim, dropout_rate=dropout_rate).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCELoss()

    # Training
    for epoch in range(epochs):
        model.train()
        for image_batch, text_batch, label_batch in train_loader:
            image_batch = image_batch.to(device)
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device)

            optimizer.zero_grad()
            output = model(image_batch, text_batch)
            loss = criterion(output.squeeze(), label_batch.float())
            loss.backward()
            optimizer.step()

    # Evaluate
    model.eval()
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for image_batch, text_batch, label_batch in val_loader:
            image_batch = image_batch.to(device)
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device)

            output = model(image_batch, text_batch)
            val_predictions.extend(output.cpu().numpy())
            val_labels.extend(label_batch.cpu().numpy())


    val_predictions = np.array(val_predictions) > 0.5


    val_acc = accuracy_score(val_labels, val_predictions)

    return val_acc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2024-11-10 05:59:46,897] A new study created in memory with name: no-name-27468581-9b23-4210-ad23-b8006c15517f
<ipython-input-17-6e0da9531eef>:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
<ipython-input-17-6e0da9531eef>:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 0.01, 0.1)
<ipython-input-17-6e0da9531eef>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rat

In [18]:
best_params = study.best_params

learning_rate = best_params['learning_rate']
batch_size = best_params['batch_size']
epochs = best_params['epochs']
weight_decay = best_params['weight_decay']
dropout_rate = best_params['dropout_rate']

print("Loaded Best Hyperparameters:", best_params)

Loaded Best Hyperparameters: {'learning_rate': 0.000122235845833759, 'batch_size': 8, 'epochs': 15, 'weight_decay': 0.064616307498029, 'dropout_rate': 0.45584099758281393}


In [19]:
import json

with open("best_params.json", "w") as f:
    json.dump(best_params, f)

with open("best_params.json", "r") as f:
    best_params = json.load(f)

In [20]:

X_train, X_val, y_train, y_val = train_test_split(image_features, labels, test_size=0.2, random_state=42)
text_train, text_val = train_test_split(text_features, test_size=0.2, random_state=42)

train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(text_train), torch.tensor(y_train))
val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(text_val), torch.tensor(y_val))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

input_dim = image_features.shape[1]
hidden_dim = 256
model = MultimodalClassifier(input_dim=input_dim, hidden_dim=hidden_dim, dropout_rate=dropout_rate).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.BCELoss()

for epoch in range(epochs):
    model.train()
    for image_batch, text_batch, label_batch in train_loader:
        image_batch = image_batch.to(device)
        text_batch = text_batch.to(device)
        label_batch = label_batch.to(device)

        optimizer.zero_grad()
        output = model(image_batch, text_batch)
        loss = criterion(output.squeeze(), label_batch.float())
        loss.backward()
        optimizer.step()

model.eval()
val_predictions = []
val_labels = []

with torch.no_grad():
    for image_batch, text_batch, label_batch in val_loader:
        image_batch = image_batch.to(device)
        text_batch = text_batch.to(device)
        label_batch = label_batch.to(device)

        output = model(image_batch, text_batch)
        val_predictions.extend(output.cpu().numpy())
        val_labels.extend(label_batch.cpu().numpy())

val_predictions = np.array(val_predictions) > 0.5
val_acc = accuracy_score(val_labels, val_predictions)

print(f"Validation Accuracy: {val_acc}")


<ipython-input-20-791d2d8b8ec9>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(text_train), torch.tensor(y_train))
<ipython-input-20-791d2d8b8ec9>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(text_val), torch.tensor(y_val))


Validation Accuracy: 1.0
